In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import cluster
from sklearn import datasets
from sklearn import mixture
from scipy.stats import multivariate_normal
from sklearn.datasets import make_spd_matrix

In [0]:
plt.rcParams["axes.grid"] = False

In [0]:
means = [[0 for i in range(32)],[1 for j in range(32)]]
mu, sigma = 0, 1 
s = np.random.normal(mu, sigma, 1)
means[1] = [s[0]*i for i in means[1]]
print(s)

[-0.39667357]


In [0]:
cov_array = []
mean_array = []
c1  = [0.5,1,1.5,2,3,4,8]

for i in range(7):
    t = []
    for j in range(2):
        t.append(np.identity(32))
    cov_array.append(t)
    
for i in c1:
    temp = means.copy()
    temp[1] = [i*j for j in temp[1]]
    mean_array.append(temp)

**Dataset Formation**

In [0]:
data_array = [] # array containing all 7 datasets
for i in range(7):
    dataset = []
    for mean, cov in zip(mean_array[i],cov_array[i]):
        x = np.random.multivariate_normal(mean, cov, 5000)
        dataset += list(x)
    data_array.append(np.array(dataset))

**EM Algorithm**

In [0]:
def EM(data,iters,clusters,i):
  ik = i
  means = None
  covars = None
  probs = None
  means = np.random.random_sample(size=(clusters, data.shape[1]))
  covars = np.array([np.identity(data.shape[1])*3 + 1e-7*np.ones(data.shape[1]) for i in range(clusters)])
  probs = np.ones(clusters)
  probs = probs/clusters

  log_likelihoods = []

  for _ in range(iters):
    # Expectation Step
    r = np.zeros(shape=(data.shape[0], clusters))

    for c in range(clusters):
      r[:, c] = probs[c] * multivariate_normal.pdf(data, mean=means[c], cov=covars[c], allow_singular=True)

    k = np.sum(r,axis=1)
    for i in range(r.shape[0]):
      if(k[i]!=0):
        r[i] = r[i]/k[i]

    # Maximisation step
    probs = np.sum(r, axis=0)/np.sum(np.sum(r, axis=0))
    temp = np.sum(r, axis=0)
    means = np.dot(np.transpose(r), data)/temp[:, None]

    for c in range(clusters):
      covars[c] = (1/temp[c]) * np.dot((r[:,c].reshape(-1,1)*(data-means[c])).T, (data-means[c]))
      
    log_likelihoods.append(np.log(np.sum([k*multivariate_normal(means[i],covars[j]).pdf(data) for k,i,j in zip(probs,range(len(means)),range(len(covars)))])))
  err = np.linalg.norm(mean_array[ik][0]-means[0])+np.linalg.norm(mean_array[ik][1] - means[1])
  print("||(μ1 − μ_1) + (μ2 − μ_2)||_2 using EM algorithm for c =", str(c1[ik]),"is: ", err)

In [0]:
for i in range (7):
  points = data_array[i]
  EM(points, 50, 2,i)

||(μ1 − μ_1) + (μ2 − μ_2)||_2 using EM algorithm for c = 0.5 is:  1.245692403764794
||(μ1 − μ_1) + (μ2 − μ_2)||_2 using EM algorithm for c = 1 is:  2.322045367986703
||(μ1 − μ_1) + (μ2 − μ_2)||_2 using EM algorithm for c = 1.5 is:  3.39345744914443
||(μ1 − μ_1) + (μ2 − μ_2)||_2 using EM algorithm for c = 2 is:  5.139794440094862
||(μ1 − μ_1) + (μ2 − μ_2)||_2 using EM algorithm for c = 3 is:  6.819965399599326
||(μ1 − μ_1) + (μ2 − μ_2)||_2 using EM algorithm for c = 4 is:  9.40027162915333
||(μ1 − μ_1) + (μ2 − μ_2)||_2 using EM algorithm for c = 8 is:  35.91235997791324


**kmeans++ Algorithm**

In [0]:
import sys
maxs = sys.maxsize

def distance(p1, p2): 
    return np.sum((p1 - p2)**2) 
   
def initialize(data, k): 
    l1 = [] 
    l1.append(data[np.random.randint(data.shape[0]), :]) 
   
    for _ in range(k - 1): 
        dist = [] 
        for i in range(data.shape[0]): 
            p = data[i, :] 
            d = maxs 
            for j in range(len(l1)): 
                temp_dist = distance(p, l1[j]) 
                d = min(d, temp_dist) 
            dist.append(d) 
    
        dist = np.array(dist) 
        n_c = data[np.argmax(dist), :] 
        l1.append(n_c) 
    return l1

def assign(d,c,k):
    clusters = [[] for i in range(k)]
    for i in d:
        dist = []
        for j in range(k):
            dist.append(distance(i,c[j]))
        temp = dist.index(min(dist))
        clusters[temp].append(i)
    return clusters

def recal(clusters):
    l = []
    for i in range (len(clusters)):
        l.append(np.mean(np.array(clusters[i])))
    return l

def kmeansalgo(d,init,iters,k):
    c = init.copy()
    for i in range(iters):
        cl = assign(d,c,k)
        updated = recal(cl)
        c = updated
    return c

In [0]:
for i in range(7):
    centroids = initialize(data_array[i], 2)
    new_centers = kmeansalgo(data_array[i],centroids,5,2)
    err1 = np.linalg.norm(new_centers[0]-mean_array[i][0])
    err2 = np.linalg.norm(new_centers[1]-mean_array[i][1])
    err = err1+err2
    print("||(μ1 − μ_1) + (μ2 − μ_2)||_2 using kmeans++ algorithm for c =", str(c1[i]),"is: ", err)

||(μ1 − μ_1) + (μ2 − μ_2)||_2 using kmeans++ algorithm for c = 0.5 is:  2.96407165194376
||(μ1 − μ_1) + (μ2 − μ_2)||_2 using kmeans++ algorithm for c = 1 is:  4.729004559934972
||(μ1 − μ_1) + (μ2 − μ_2)||_2 using kmeans++ algorithm for c = 1.5 is:  0.08414478295809401
||(μ1 − μ_1) + (μ2 − μ_2)||_2 using kmeans++ algorithm for c = 2 is:  8.996706644727656
||(μ1 − μ_1) + (μ2 − μ_2)||_2 using kmeans++ algorithm for c = 3 is:  0.028958203027592312
||(μ1 − μ_1) + (μ2 − μ_2)||_2 using kmeans++ algorithm for c = 4 is:  17.963455683652413
||(μ1 − μ_1) + (μ2 − μ_2)||_2 using kmeans++ algorithm for c = 8 is:  0.009299929476780044
